In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df = pd.read_csv('data_onevisa_postprocess.csv').set_index('sample_id')
df.shape

In [ ]:
summary = pd.DataFrame({
        "number_unique": df.nunique(),
        "number_nan": df.isna().sum(),
        "dtype_df": df.dtypes
    })

In [ ]:
df['generated'] = df['generated'].apply(lambda x: x.split(',')[1])

In [ ]:
df['generated_date'] = pd.to_datetime(df['generated'], dayfirst=True)


In [ ]:
list_columns_remove = summary[(summary.number_unique>df.shape[0]*0.8) & (summary.dtype_df=='object')].sort_values(by='number_unique').index.tolist()
list_columns_remove_f = [i for i in list_columns_remove if 'date' not in i]

In [ ]:
df_f = df.drop(columns = list_columns_remove_f)

In [ ]:
df_f.shape

In [ ]:
date_cols = [c for c in df.columns if 'date' in c.lower()]

# 2) Compute their missing‐value percentages
miss_pct = df_f[date_cols].isna().mean()

# 3) Drop those over 80% missing
to_drop = miss_pct[miss_pct > 0.8].index
df_f = df_f.drop(columns=to_drop)
print(df_f.shape)

In [ ]:
df_f = df_f[df_f['date_of_issue_nic']!='Place of birth']
df_f = df_f[df_f['date_of_expiry_nic']!='Place of birth']

df_f = df_f.drop(columns = ['date_from_e_pd','date_to_e_pd','date_from_cv',
                            'date_to_cv','date_from_ehd','date_to_ehd','date_from_e_scd',
                            'date_to_e_scd'])

In [ ]:
date_cols = [
    c for c in df_f.columns
    if 'date' in c.lower() and c != 'generated_date']

for col in date_cols:
    print(col)
    # convert the column to datetime if it isn’t already
    df_f[col] = pd.to_datetime(df_f[col], dayfirst=True)
    # replace with difference in days (you can keep the Timedelta if you prefer)
    df_f[col] = (df_f['generated_date'] - df_f[col]).dt.days

In [ ]:
today = pd.Timestamp.today().normalize()
df_f['days_from_today'] = (today - df_f['generated_date']).dt.days

df_f = df_f.drop(columns= ['generated_date'])

In [ ]:
df_f = df_f[df_f.in_australia_x!='Character declarations']

In [ ]:
df_f.shape

In [ ]:
summary = pd.DataFrame({
        "number_unique": df_f.nunique(),
        "number_nan": df_f.isna().sum(),
        "dtype_df": df_f.dtypes
    })

In [ ]:
df_f.type_case.value_counts()

In [ ]:
summary[summary.index.isin([i for i in df_f.columns if 'australia' in i])]

In [ ]:
df_f[df_f.stay_in_australia_itehohcf=='Character declarations']

In [ ]:
df_f.stay_in_australia_itehohcf.unique()

In [ ]:
df_f.shape

In [ ]:
df_f.to_csv('data_onevisa_postprocess_v2.csv', index=True)